In [1]:
!git clone https://github.com/corydambach/mmrotate-sandbox

Cloning into 'mmrotate-sandbox'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 55 (delta 15), reused 52 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 273.91 KiB | 22.83 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
%cd mmrotate-sandbox/

/content/mmrotate-sandbox


In [3]:
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 "numpy<2" --index-url https://download.pytorch.org/whl/cu118
!pip install -q -U openmim "numpy<2"
# Install MMCV compiled for PyTorch 2.0.1 (not 2.1.0)
!pip install -q mmcv-full==1.7.2 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html "numpy<2"
!mim install -q mmdet\<3.0.0
!pip uninstall -q mmrotate --yes
!rm -rf mmrotate
!git clone -q https://github.com/open-mmlab/mmrotate.git
!cd mmrotate && pip install -q -e .
!mkdir model
!wget -q -O ./model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth https://download.openmmlab.com/mmrotate/v0.1.0/redet/redet_re50_fpn_1x_dota_ms_rr_le90/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 97.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but yo

In [4]:
exit()

In [3]:
%cd mmrotate-sandbox/

[Errno 2] No such file or directory: 'mmrotate-sandbox/'
/content/mmrotate-sandbox


In [4]:

#os
import os

#ml
import numpy as np
import torch
import mmcv
import cv2
from mmdet.apis    import init_detector
from mmrotate.apis import inference_detector_by_patches
from dataclasses import asdict
import json

from geo_util import Pnt, Vehicle, VehicleExport, rbbox_to_poly


In [5]:

def process_results( result, conf_thr: float ):
    if isinstance(result, tuple):
        bbox_result, segm_result = result
        if isinstance( segm_result, tuple ):
            segm_result = segm_result[0]
    else:
        bbox_result, segm_result = result, None
    bboxes = np.vstack(bbox_result)
    labels = [
        np.full( bbox.shape[0], i, dtype=np.int32 )
        for i, bbox in enumerate(bbox_result)
    ]
    labels = np.concatenate( labels )
    #remove everything that isn't a small vehicle
    # Apply both filters together
    conf_mask = bboxes[:, 5] >= conf_thr
    class_mask = (labels == 4) | (labels == 5)
    combined_mask = conf_mask & class_mask
    bboxes = bboxes[combined_mask]
    labels = labels[combined_mask]
    # print( f"Detected {len( bboxes )} vehicles." )
    reshaped = labels.reshape( -1, 1 )
    concat = np.concatenate( (bboxes, reshaped), axis=1 )
    a_entries = [Vehicle( r[0], r[1], r[2], r[3], r[4], r[5], r ) for r in concat]
    vas = [VehicleExport( Pnt( a.x, a.y ), a.width, a.height, a.theta, a.confidence, rbbox_to_poly( a.arr ).tolist(), str(i) ) for i, a in enumerate(a_entries)]
    return vas



In [6]:

class InitArgs:
    def __init__(self):
        self.device     = 'cuda:0'
        self.config     = "mmrotate/configs/redet/redet_re50_refpn_1x_dota_ms_rr_le90.py"
        self.checkpoint = "model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth"

class Args:
    def __init__( self, batch_size: int, patch_size: int, patch_step: int, img: str, conf: float ):
        self.img = img
        self.score_thr = conf
        self.merge_iou_thr = 0.85
        self.img_ratios = [1.0]
        self.batch_size = batch_size
        self.patch_sizes = [patch_size]
        self.patch_steps = [patch_step]
        self.palette = 'dota'

torch.cuda.set_per_process_memory_fraction( 1.0 )
init_args = InitArgs()



In [7]:

try:
    model
except NameError:
    model = init_detector( init_args.config, init_args.checkpoint, device=init_args.device )


/content/mmrotate-sandbox/mmrotate/mmrotate/models/backbones/re_resnet.py:481: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/usr/local/lib/python3.11/dist-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)
/usr/local/lib/python3.11/dist-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: model/redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.conv1.filter, backbone.layer2.0.conv1.filter, backbone.layer2.0.conv2.filter, backbone.layer2.0.conv3.filter, backbone.layer2.0.downsample.0.filter, backbone.layer2.1.conv1.filter, backbone.layer2.1.conv2.filter, backbone.layer2.1.conv3.filter, backbone.layer2.2.conv1.filter, backbone.layer2.2.conv2.filter, backbone.layer2.2.conv3.filter, backbone.layer2.3.conv1.filter, backbone.layer2.3.conv2.filter, backbone.layer2.3.conv3.filter, backbone.layer3.0.conv1.filter, backbone.layer3.0.conv2.filter, backbone.layer3.0.conv3.filter, backbone.layer3.0.downsample.0.filter, backbone.layer3.1.conv1.filter, backbone.layer3.1.conv2.filter, backbone.layer3.1.conv3.filter, backbone.layer3.2.conv1.filter, backbone.layer3.2.conv2.filter, backbone.layer3.2.conv3.filter, backbone.layer3.3.conv1.filter, backbone.layer3.3.co

In [8]:
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image

def load_gt(img_path: str) -> List[VehicleExport]:
    """Load ground truth from corresponding .txt file"""
    gt_path = Path(img_path).with_suffix('.txt')
    if not gt_path.exists():
        print(f"Warning: GT file {gt_path} not found")
        return []

    # Get image dims with fallback
    try:
        w, h = Image.open(img_path).size
    except:
        w, h = 1024, 1024
        print(f"Warning: Using default dims for {img_path}")

    vehicles = []
    with open(gt_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6:
                continue

            cls, cx_n, cy_n, w_n, h_n, angle = map(float, parts[:6])
            cx, cy, w_abs, h_abs = cx_n * w, cy_n * h, w_n * w, h_n * h

            arr = np.array([cx, cy, w_abs, h_abs, angle, 1.0, cls])
            poly = rbbox_to_poly(arr).tolist()

            vehicles.append(VehicleExport(
                label=int(cls), center=Pnt(cx, cy), width=w_abs,
                height=h_abs, theta=angle, polygon=poly, confidence=1.0
            ))

    return vehicles

def get_dist(p1, p2):
    """Euclidean distance between points"""
    return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

def find_matches(dets: List[VehicleExport], gt: List[VehicleExport], max_dist: float = 10.0):
    """Find nearest GT matches using greedy assignment"""
    matches = []
    used = set()

    for i, det in enumerate(dets):
        best_dist, best_j = float('inf'), -1

        for j, g in enumerate(gt):
            if j in used:
                continue
            dist = get_dist(det.center, g.center)
            if dist < best_dist and dist <= max_dist:
                best_dist, best_j = dist, j

        if best_j != -1:
            matches.append((i, best_j, best_dist))
            used.add(best_j)

    return matches



In [9]:

def benchmark(img_path: str, dets: List[VehicleExport]):
    """Simple benchmark using nearest center matching"""
    gt = load_gt(img_path)
    if not gt:
        print(f"No GT for {Path(img_path).name}")
        return

    matches = find_matches(dets, gt, 10)

    tp = len(matches)
    fp = len(dets) - tp
    fn = len(gt) - tp

    prec = tp / len(dets) if dets else 0.0
    rec = tp / len(gt) if gt else 0.0
    f1 = 2 * prec * rec / (prec + rec) if prec + rec else 0.0

    name = Path(img_path).name
    print(f"\n{name}:")
    print(f"Dets: {len(dets)}, GT: {len(gt)}")
    print(f"Prec: {prec:.3f}, Rec: {rec:.3f}, F1: {f1:.3f}")
    print(f"TP: {tp}, FP: {fp}, FN: {fn}")

    return {
        'precision': prec, 'recall': rec, 'f1': f1,
        'tp': tp, 'fp': fp, 'fn': fn, 'matches': matches
    }


In [ ]:

import pathlib as path
import glob

dir  = "./input"
imgs = txt_files = glob.glob(  f"{dir}/*.png" )
for img_path in imgs:
    args = Args( batch_size=32, patch_size=384, patch_step=64, img=img_path, conf=0.00 )
    img = mmcv.imread( args.img )
    result = inference_detector_by_patches( model, img,
                                            args.patch_sizes,
                                            args.patch_steps,
                                            args.img_ratios,
                                            args.merge_iou_thr,
                                            args.batch_size )

    dets = process_results( result, args.score_thr )
    benchmark( img_path, dets )
    # model.show_result( img, result, show=True )
    #print( dets )

/usr/local/lib/python3.11/dist-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '



29.png:
Dets: 258, GT: 200
Prec: 0.729, Rec: 0.940, F1: 0.821
TP: 188, FP: 70, FN: 12

18.png:
Dets: 55, GT: 38
Prec: 0.636, Rec: 0.921, F1: 0.753
TP: 35, FP: 20, FN: 3

31.png:
Dets: 34, GT: 29
Prec: 0.765, Rec: 0.897, F1: 0.825
TP: 26, FP: 8, FN: 3

9.png:
Dets: 128, GT: 86
Prec: 0.625, Rec: 0.930, F1: 0.748
TP: 80, FP: 48, FN: 6
